In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization, Dropout
from keras import optimizers
from keras import regularizers
import requests
import pprint
from enum import Enum
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


Using TensorFlow backend.


In [3]:

class Attributes(Enum):
    PACE = 'pace'
    SHOT = 'shot'
    PASS = 'pass'
    DRIBBLE = 'dribble'
    DEFENSE = 'defense'
    PHYSICAL = 'physical'
    
class Positions(Enum):
    ATTACKER = 1
    LEFT_WING = 2
    RIGHT_WING = 3
    CENTER_MID = 4
    DEFENSIVE_MID = 5
    CENTER_BACK = 6
    FULL_BACK = 7
    KEEPER = 8

n_pages = 180

In [4]:
data = []
for i in range(1, n_pages):
    response = requests.get('https://www.easports.com/fifa/ultimate-team/api/fut/item?page={}'.format(i))
    json = response.json()
    data.append(json)

items = [item['items'] for item in data]
items = [item for sublist in items for item in sublist]

In [5]:
players = []
for item in items:
    player = dict()
    if not item['commonName']:
        player.update({'name': item['firstName'] + ' ' + item['lastName']})
    else:
        player.update({'name': item['commonName']})

    all_attributes = []
    for attribute in item['attributes']:
        attributes = dict()
        attribute_name = attribute['name']
        attribute_value = attribute['value']
        attributes['value'] = attribute_value
        if attribute_name == 'fut.attribute.PAC':
            attributes['name'] = Attributes.PACE.name
        elif attribute_name == 'fut.attribute.SHO':
            attributes['name'] = Attributes.SHOT.name
        elif attribute_name == 'fut.attribute.PAS':
            attributes['name'] = Attributes.PASS.name
        elif attribute_name == 'fut.attribute.DRI':
            attributes['name'] = Attributes.DRIBBLE.name
        elif attribute_name == 'fut.attribute.DEF':
            attributes['name'] = Attributes.DEFENSE.name
        elif attribute_name == 'fut.attribute.PHY':
            attributes['name'] = Attributes.PHYSICAL.name
        all_attributes.append(attributes)
    
    player['attributes'] = all_attributes
    player['rating'] = item['rating']
    if item['position'] == 'ST' or item['position'] == 'CF':
        player['position'] = Positions.ATTACKER.value
    elif item['position'] == 'LW' or item['position'] == 'LM' or item['position'] == 'LF':
        player['position'] = Positions.LEFT_WING.value
    elif item['position'] == 'RW' or item['position'] == 'RM' or item['position'] == 'RF':
        player['position'] = Positions.RIGHT_WING.value
    elif item['position'] == 'CM' or item['position'] == 'CAM':
        player['position'] = Positions.CENTER_MID.value
    elif item['position'] == 'CDM':
        player['position'] = Positions.DEFENSIVE_MID.value
    elif item['position'] == 'CB':
        player['position'] = Positions.CENTER_BACK.value
    elif item['position'] == 'LB' or item['position'] == 'RB' or item['position'] == 'LWB' or item['position'] == 'RWB':
        player['position'] = Positions.FULL_BACK.value
    elif item['position'] == 'GK':
        player['position'] = Positions.KEEPER.value
    players.append(player)
            

    

In [6]:
player_data = []
labels = []
for i in range(len(players)-100):
    data = []
    for attribute in players[i]['attributes']:
        data.append(attribute['value'])
    data.append(players[i].get('position'))
    player_data.append(data)
    labels.append(players[i]['rating'])
    
# player_data = preprocessing.normalize(player_data)
player_data = np.asarray(player_data)

data = []
maximum = np.max(player_data)
minimum = np.min(player_data)

for player in player_data:
    normed = (player-minimum)/(maximum-minimum)
    data.append(normed)
data = np.array(data)

labels = np.asarray(labels)
training_x, testing_x, training_y, testing_y = train_test_split(player_data, labels, test_size=0.25, random_state=42)

In [7]:
#  FOR UNNORMED DATA
# model = Sequential()
# model.add(Dense(32, input_shape=(7,)))
# model.add(Activation('relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(28, activation='relu'))
# model.add(Dense(20, activation='relu'))
# model.add(Dense(28, activation='relu'))
# model.add(Dense(20, activation='relu'))
# model.add(Dense(1, activation='linear'))

# adam = optimizers.Adam(lr=0.0018, beta_1=0.91, beta_2=0.99, epsilon=None, decay=2e-6, amsgrad=True)
# model.compile(loss='mean_squared_error', metrics=['accuracy'], optimizer=adam)
# model.fit(training_x, training_y, epochs=70, batch_size=16, shuffle=True, verbose=2)


model = Sequential()
model.add(Dense(32, input_shape=(7,)))
model.add(Activation('relu'))
model.add(Dense(28, activation='relu'))
model.add(Dense(28, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))

adam = optimizers.Adam(lr=0.0018, beta_1=0.91, beta_2=0.99, epsilon=None, decay=2e-6, amsgrad=True)
model.compile(loss='mean_squared_error', metrics=['accuracy'], optimizer=adam)
model.fit(training_x, training_y, epochs=70, batch_size=16, shuffle=True, verbose=2)
# model = Sequential()
# model.add(Dense(64, input_shape=(7,)))
# model.add(Activation('relu'))
# # model.add(Dropout(0.5))
# model.add(Dense(16, activation='tanh'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(32, activation='relu'))
# # model.add(Dense(64, activation='relu'))
# # model.add(Dense(20, activation='relu'))
# model.add(Dense(1, activation='linear'))
# #sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.99, nesterov=True)
# # sgd = optimizers.RMSprop(lr=0.1, rho=0.95, epsilon=None, decay=0.001)
# # 
# adam = optimizers.Adam(lr=0.0017, beta_1=0.9, beta_2=0.99, epsilon=None, decay=2e-6, amsgrad=True)
# model.compile(loss='mean_squared_error', metrics=['accuracy'], optimizer=adam)
# model.fit(training_x, training_y, epochs=70, batch_size=16, shuffle=True, verbose=2)

Epoch 1/70
 - 29s - loss: 242.6382 - acc: 0.0896
Epoch 2/70
 - 1s - loss: 6.4171 - acc: 0.1605
Epoch 3/70
 - 1s - loss: 4.3795 - acc: 0.2040
Epoch 4/70
 - 1s - loss: 3.5927 - acc: 0.2285
Epoch 5/70
 - 1s - loss: 3.3107 - acc: 0.2304
Epoch 6/70
 - 1s - loss: 3.0357 - acc: 0.2485
Epoch 7/70
 - 1s - loss: 2.8399 - acc: 0.2558
Epoch 8/70
 - 1s - loss: 2.8775 - acc: 0.2558
Epoch 9/70
 - 1s - loss: 2.8022 - acc: 0.2558
Epoch 10/70
 - 1s - loss: 2.6980 - acc: 0.2691
Epoch 11/70
 - 1s - loss: 2.6198 - acc: 0.2758
Epoch 12/70
 - 1s - loss: 2.5591 - acc: 0.2825
Epoch 13/70
 - 1s - loss: 2.5334 - acc: 0.2850
Epoch 14/70
 - 1s - loss: 2.5674 - acc: 0.2761
Epoch 15/70
 - 1s - loss: 2.5941 - acc: 0.2755
Epoch 16/70
 - 1s - loss: 2.4467 - acc: 0.2831
Epoch 17/70
 - 1s - loss: 2.4003 - acc: 0.2898
Epoch 18/70
 - 1s - loss: 2.4533 - acc: 0.2847
Epoch 19/70
 - 1s - loss: 2.3425 - acc: 0.2968
Epoch 20/70
 - 1s - loss: 2.3842 - acc: 0.2882
Epoch 21/70
 - 1s - loss: 2.4518 - acc: 0.2838
Epoch 22/70
 - 1s -

In [8]:
predict = model.predict(testing_x, verbose=1)
for i in range(predict.shape[0]):
    print('predicted: {0}, actual: {1}'.format(predict[i][0], testing_y[i]))

model.save("fut.h5")

1049/1049 [==============================] - 0s 77us/step
predicted: 89.2634506225586, actual: 91
predicted: 79.08692169189453, actual: 79
predicted: 88.7427749633789, actual: 87
predicted: 76.56150817871094, actual: 76
predicted: 77.3951416015625, actual: 77
predicted: 82.05747985839844, actual: 82
predicted: 90.34986877441406, actual: 90
predicted: 79.98066711425781, actual: 79
predicted: 79.60836029052734, actual: 81
predicted: 89.5822982788086, actual: 90
predicted: 82.08172607421875, actual: 83
predicted: 83.4117202758789, actual: 82
predicted: 75.21321105957031, actual: 75
predicted: 89.50454711914062, actual: 91
predicted: 82.09882354736328, actual: 82
predicted: 81.72254943847656, actual: 81
predicted: 80.63175201416016, actual: 78
predicted: 87.5078125, actual: 87
predicted: 80.57049560546875, actual: 82
predicted: 89.65666961669922, actual: 89
predicted: 84.7748031616211, actual: 84
predicted: 85.83078002929688, actual: 87
predic

In [ ]:
pp = pprint.PrettyPrinter(indent=2)
for player in players:
    pp.pprint(player)